In [1]:
import os
import math
import gc

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
from nltk.stem.snowball import PorterStemmer

# Arquivos Pythons com funções e constantes
from qs_constants import AppConstants
from qs_functions import load_qa_pairs, normalize_qa, remove_duplication, remove_bad_answer, get_context_files
from qs_classes import TitleGroup, QuestionEmbedding

import tensorflow as tf

from transformers import AutoTokenizer, AutoModel

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as t_funcional

from tqdm import tqdm

In [2]:
nlp = spacy.load('en_core_web_lg')
stemmer = PorterStemmer()

In [3]:
def carrega_dataset():
    df_all = load_qa_pairs()
    df_preparado = df_all[(df_all.Question.notna()) &
                          (df_all.Answer.notna())]
    vet_QA = df_preparado[["Question", "Answer", "ArticleTitle","ArticleFile"]].to_numpy()
    vet_QA = normalize_qa(vet_QA, lowercase=False)
    vet_QA = remove_bad_answer(vet_QA)
    vet_QA = remove_duplication(vet_QA)
    return vet_QA


In [4]:
def agrupa_perguntas_por_assunto(vet_QA):
    qa_agrupada_assunto = {}
    for qa in vet_QA:
        assunto = qa[AppConstants.COL_TITULO]
        assunto_preparado = assunto.lower().replace("_", " ")

        if assunto_preparado not in qa_agrupada_assunto:
            grupo = TitleGroup()
            qa_agrupada_assunto[assunto_preparado] = grupo
        else:
            grupo = qa_agrupada_assunto[assunto_preparado]

            
        q_embedding = QuestionEmbedding()
        q_embedding.question = qa[AppConstants.COL_PERGUNTA]
        q_embedding.answer = qa[AppConstants.COL_RESPOSTA]
        
        grupo.question_list.append(q_embedding)

        # existem arquivos com nan
        file = qa[AppConstants.COL_CONTEXT_FILE]
        if isinstance(file, str):
            grupo.files.add(file)
    return qa_agrupada_assunto
    
    

In [5]:
def carrega_contexto_assunto(qa_agrupada_assunto):
    with  tqdm(total=len(qa_agrupada_assunto), position=0, desc='Carregando Assuntos...') as progress:
        for k_grupo in qa_agrupada_assunto:    
            progress.set_description(f"Carregando Assunto [{k_grupo:<30}]")
            grupo = qa_agrupada_assunto[k_grupo]
            for ctx_file in grupo.files:
                file_path = os.path.join(AppConstants.CONTEXT_TEXT_PATH, ctx_file + AppConstants.CONTEXT_FILE_EXTENSION)
                with open(file_path, 'r', encoding='utf-8') as f_ctx:
                    ctx_file_doc = f_ctx.read()
                    ctx_nlp_doc = nlp(ctx_file_doc)
                    if ctx_nlp_doc.ents:
                        for ent in ctx_nlp_doc.ents:
                            entidade = ent.text
                            grupo.context_ner.add(entidade)
            
            progress.update(1)
        progress.set_description("Assuntos carregados.")

In [6]:
# retorna uma lista de asssuntos que tem potencial match com a pergunta
def get_assunto_pergunta(qa_agrupada_assunto, pergunta):
    doc_pergunta = nlp(pergunta)
    lista_assunto_match = []
    
    for k_assunto in qa_agrupada_assunto:
        grupo = qa_agrupada_assunto[k_assunto]
        achou = False
        como_achou = ''
        
        # NER pergunta
        if doc_pergunta.ents:
            for entidade in doc_pergunta.ents:
                entidade_text = entidade.text.lower()
                for entidade_token in entidade_text.split():
                    if entidade_token in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
                        
        # Prcura por nomes
        if not achou and doc_pergunta.noun_chunks:
            for noun in doc_pergunta.noun_chunks:
                for n_part in noun.text.split():
                    if n_part in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA    
    
         # Lemmanization
        if not achou:
            doc_assunto = nlp(k_assunto)
            for token_assunto in doc_assunto:
                if token_assunto.is_stop:
                    continue
                assunto_lemma = token_assunto.lemma_.lower()
                for token_pergunta in doc_pergunta:
                    if token_pergunta.is_stop:
                        continue
                    if assunto_lemma == token_pergunta.lemma_.lower():
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
                if not achou:
                    for token_pergunta in doc_pergunta:
                        if token_pergunta.is_stop:
                            continue
                        if stemmer.stem(token_pergunta.text) == stemmer.stem(token_assunto.text):
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_STEMMER    
                    
        # NER context file            
        if not achou:
            if doc_pergunta.ents:
                for entidade in doc_pergunta.ents:
                    entidade_text = entidade.text
                    if entidade_text in grupo.context_ner:
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
                            
        
        if achou:
            lista_assunto_match.append((k_assunto, como_achou))
            
    return lista_assunto_match

In [7]:
# dada uma lista de matchs para assunto retonra aquele que teoricamente seria o de maior peso, seguindo a ordem de prioridade por busca
def get_assunto_match_principal(lista_assunto_match):
    
    def encontra_metodo_busca(lista_assunto, metodo_busca_pesquisa):
        for (assunto, metodo_busca) in lista_assunto_match:
            if metodo_busca == metodo_busca_pesquisa:
                return assunto
        return None
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
        
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
    
    

In [16]:
# imprime as perguntas cuja a entidade não conseguiu ser encontrada
def teste_busca_contexto():
    vet_QA = carrega_dataset()
    qa_agrupada_assunto = agrupa_perguntas_por_assunto(vet_QA)
    carrega_contexto_assunto(qa_agrupada_assunto)

    total = 0
    qtd_achou_lista = 0
    qtd_achou_lista_errada = 0
    qtd_achou_unico = 0
    qtd_achou_unico_lista = 0
    qtd_nao_achou = 0
    qtd_achou_unico_incorreto = 0


    estatistica_busca = {AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA: 0, 
                         AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_STEMMER : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO : 0,
                         'PERGUNTA_SEM_CONTEXTO' : 0
                        }

    for k_assunto in qa_agrupada_assunto:
        grupo = qa_agrupada_assunto[k_assunto]
        for qa in grupo.question_list:
            total += 1
            achou = False
            como_achou = ""

            pergunta = qa.question

            lista_assunto_match = get_assunto_pergunta(qa_agrupada_assunto, pergunta)

            if len(lista_assunto_match) == 1:
                (assunto_encontrado, metodo_busca) = lista_assunto_match[0]
                if assunto_encontrado != k_assunto:
                    qtd_achou_unico_incorreto += 1
                else:
                    qtd_achou_unico += 1
                    estatistica_busca[metodo_busca] += 1

            elif len(lista_assunto_match) > 0:
                assunto_encontrado, metodo_busca = get_assunto_match_principal(lista_assunto_match)
                if assunto_encontrado == k_assunto:
                    qtd_achou_unico_lista += 1
                    estatistica_busca[metodo_busca] += 1
                else:
                    achou_lista = False
                    for (assunto_encontrado, metodo_busca) in lista_assunto_match:
                        if assunto_encontrado != k_assunto:
                            continue
                        achou_lista = True
                    if achou_lista:
                        qtd_achou_lista += 1
                    else:
                        qtd_achou_lista_errada += 1
            else:
                print("Não achou ", k_assunto," - ", pergunta)
                qtd_nao_achou +=1
                for token_sem_ctx in AppConstants.TOKENS_COM_FALTA_CONTEXTO:
                    if token_sem_ctx in pergunta:
                        estatistica_busca['PERGUNTA_SEM_CONTEXTO'] += 1


    print ('Total:', total)
    print('Achou Unico', qtd_achou_unico)
    print('Achou Unico Pela Lista através de prioridade entre métodos ', qtd_achou_unico_lista)
    print('Achou Unico Incorreto', qtd_achou_unico_incorreto)
    print('Achou Lista ', qtd_achou_lista)
    print('Achou Lista Errada ', qtd_achou_lista_errada)
    print('Não Achou', qtd_nao_achou)
    print(estatistica_busca)

In [ ]:
teste_busca_contexto()

### Processando as Sentenças para geração de Embedding

In [8]:
# criação modelo, caminho Hugginface recebido por parametro
def create_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.to('cuda')
    return model, tokenizer

In [9]:
# liberar espaço ocupado pelo modelo na GPU
def destroy_model(model):
    del model
    gc.collect()
    torch.cuda.empty_cache()

##### Prepara o dataset para entrar no modelo bert. Gera input_id, attention_mask e monta o data loaader para processamento em mini-batchs
- *Vetor de perguntas - Numpy*


In [10]:
def prepare_to_bert(np_sentences, tokenizer, sentence_max_length=450, batch_size=5):
    inputs = tokenizer.batch_encode_plus(np_sentences, 
                                         return_tensors='pt',
                                         padding='max_length',
                                         max_length=sentence_max_length)
    
    input_ids = inputs['input_ids']
    input_ids = input_ids.to('cuda')
    attention_mask = inputs['attention_mask']
    attention_mask = attention_mask.to('cuda')
    
    dataset = TensorDataset(input_ids, attention_mask)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    return loader   
    

##### Realiza o encode da sentença, retornando mean e mean polling*

In [11]:
def encode_sentence(sentences, bert_model, bert_tokenizer):
    sentence_loader = prepare_to_bert(sentences, bert_tokenizer, sentence_max_length=450, batch_size=5)

    mean_pooled_vet = []
    mean_vet = []
    
    for batch_id, (input_ids, attention_masks) in enumerate(sentence_loader):
        output = bert_model(input_ids, attention_mask=attention_masks)

        embeddings = output.last_hidden_state
        embeddings.detach()

        # calculo mean_pooling
        mask = attention_masks.unsqueeze(-1).expand(embeddings.size()).float()
        masked_embeddings = embeddings * mask
        summed = torch.sum(masked_embeddings, 1)
        summed_mask = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / summed_mask
        # convert from PyTorch tensor to numpy array
        mean_pooled = mean_pooled.cpu().data.numpy()
        mean_pooled_vet.append(mean_pooled)

        #calculo mean
        mean = embeddings.mean(dim=1)
        mean = mean.cpu().data.numpy()
        mean_vet.append(mean)

        #--- libera recursos da gpu
        del output
        gc.collect()
        torch.cuda.empty_cache()

    mean_pooled_vet = np.concatenate(mean_pooled_vet)
    mean_vet = np.concatenate(mean_vet)
    
    return mean_pooled_vet, mean_vet

In [12]:
# dada a base agrupada gera os embeddings por pergunta do contexto
def gera_embedding_pergunta(qa_agrupada_assunto):
    model, tokenizer = create_model(AppConstants.MODEL_BASE_NLI_MEAN)

    with  tqdm(total=len(qa_agrupada_assunto), position=0, desc='Encoding de Sentenças') as progress:
        for k_grupo in qa_agrupada_assunto:
            progress.set_description(f"Gerando Embedding Sentenças [{k_grupo:<30}]")
            grupo = qa_agrupada_assunto[k_grupo]
            vet_pergunta = []
            for q_embedding in grupo.question_list:
                vet_pergunta.append(q_embedding.question)
            sentence_embeddings = encode_sentence(vet_pergunta, bert_model=model, bert_tokenizer=tokenizer)    
            for index, q_embedding in enumerate(grupo.question_list):
                q_embedding.embedding = sentence_embeddings[index]
            progress.update(1)
        progress.set_description("Geração Embedding Finalizada")
    destroy_model(model_sbert_sem_finetunnig)     
    

## Execução

In [13]:
vet_QA = carrega_dataset()
qa_agrupada_assunto = agrupa_perguntas_por_assunto(vet_QA)
carrega_contexto_assunto(qa_agrupada_assunto)
gera_embedding_pergunta(qa_agrupada_assunto)
    


Assuntos carregados.: 100%|██████████████████████████████████████████████████████████| 105/105 [01:30<00:00,  1.16it/s]
Gerando Embedding Sentenças [abraham lincoln               ]:   0%|                            | 0/105 [00:01<?, ?it/s]


IndexError: tuple index out of range